## 사전설치
* colab
* korbert 모델을 사전에 설치하는 작업이 있습니다.
* colab에서 !git clone https://github.com/monologg/KoBERT-NER 을 실행하면 drive폴더와 같은 위치에 KoBERT-NER 폴더가 생성됩니다.
* KoBERT-NER폴더를 google-drive에 옮긴 다음 폴더의 이름을 바꾸는 작업이 필요합니다.

In [ ]:
!pip install transformers
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

# 설치 후 원하는 google-drive 폴더에 저장
# !git clone https://github.com/monologg/KoBERT-NER 

import os
# 저장한 위치를 지정
path = '/content/drive/My Drive/Colab Notebooks/'
 

In [ ]:
filename = 'KoBERT-NER'
cName = 'KoBERT_NER'

# kobert호출 경로 지정
os.chdir(path)


# 처음 kobert를 설치할 때만 실행
# KoBERT-NER 폴더 이름을 KoBERT_NER 으로 변경하는 작업
#os.rename(path+filename, path+cName)


In [ ]:
!pip install sentencepiece
import logging
import unicodedata
from shutil import copyfile
import sentencepiece as spm
from transformers import PreTrainedTokenizer

from transformers import BertModel
from KoBERT_NER.tokenization_kobert  import KoBertTokenizer

In [ ]:
# 모델과 토크나이저를 호출
model = BertModel.from_pretrained('monologg/kobert')
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') 


In [ ]:
# 사전에 구글드라이브에 데이터를 올려놔야함
train = pd.read_csv("train_data.txt", sep='?',encoding = 'euc-kr')
test = pd.read_csv("test_data.txt",sep='?',encoding = 'euc-kr')

df = pd.concat([train,test]).reset_index()
df = df.iloc[:,1:]

In [ ]:
import re
df['발화개요'] = df['발화개요'].str.replace(r'[=+,#/\?:^$.@*\"※~&%ㆍ!』○\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
df['발화개요'] = df['발화개요'].str.replace(r'[남,여] [0-9]{1,2}세','',regex=True)
df['발화개요'] = df['발화개요'].str.replace(r'[남,여][0-9]{1,2}세','',regex=True)
df['발화개요'] = df['발화개요'].str.replace(r'[남,여] [0-9]{1,2}년생','',regex=True)
df['발화개요'] = df['발화개요'].str.replace(r'[남,여][0-9]{1,2}년생','',regex=True)
df['발화개요'] = df['발화개요'].str.replace('  ','',regex=True)


# 데이터 크기가 35보다 적은 것들 탐색
drop_index = df[df['발화개요'].str.len() < 18].index
df = df.drop(drop_index)
labels = df['화재유형'].values



In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state = 5) 

In [1]:
# 문장을 bert 학습용 데이터 형태 [토큰, mask, segment], targer으로 분류
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 512 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # 문장 토큰화 및 padding
        token = tokenizer.encode(data_df.iloc[i,2], max_length=SEQ_LEN, pad_to_max_length=True)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(0~5)을 targets 변수에 저장해 줌
        targets.append(data_df.iloc[i,1])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df['발화개요'] = data_df['발화개요'].astype(str)
    data_df['화재유형'] = data_df['화재유형'].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 512
BATCH_SIZE = 20


# train 데이터를 버트 인풋에 맞게 변환
# train_x = [tokens, masks, segments] , train_y = targets
train_x, train_y = load_data(train)

NameError: name 'train' is not defined

In [ ]:
# tpu 사용
esolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
def create_sentiment_bert():
  # 버트 pretrained 모델 로드
    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
    token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
    mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
    segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
    bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

    bert_outputs = bert_outputs[1]
    sentiment_first = tf.keras.layers.Dense(6, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02))(bert_outputs)
    sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
  # 옵티마이저는 간단하게 Adam 옵티마이저 활용
  # 123456 # 5e-6에서는 0.56에서 정확도가 올라가지 않음
    sentiment_model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['sparse_categorical_accuracy'])
    return sentiment_model

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
# TPU 사용해서 모델학습 및 저장
# tpu 사용하지 않을 시 with strategy.scope():를 제거
with strategy.scope():
  
    sentiment_model = create_sentiment_bert()
    sentiment_model.fit(train_x, train_y, epochs=3, shuffle=False, batch_size=20)
    sentiment_model.save_weights(os.path.join(path,"sentiment_model.h5"))
      

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df.iloc[i,2 ], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]
SEQ_LEN = 512
DATA_COLUMN = 'data'
# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df['발화개요'] = data_df['발화개요'].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x
test_set = predict_load_data(test)

In [ ]:
import sys
mod = sys.modules[__name__]
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
    sentiment_model = create_sentiment_bert()
  
    sentiment_model.load_weights(os.path.join(path,"sentiment_model.h5"))
    setattr(mod, 'model', sentiment_model)
    setattr(mod, 'pred_', sentiment_model.predict(test_set, batch_size=1))

In [ ]:
def mean_answer_label(*preds):
    preds_sum = np.zeros(preds[0].shape[0])
    for pred in preds:
        preds_sum += np.argmax(pred, axis=-1)
    return np.round(preds_sum/len(preds), 0).astype(int) 

In [ ]:
pretict = mean_answer_label(pred_)

In [ ]:

def test_check(data):
    sum = 0
    for i in range(len(data)):
        if data.iloc[i,1] == pretict[i]:
            sum += 1
    return sum/len(data)



In [ ]:
test_check(test)
pretict = list(pretict)
ans = test.iloc[:,1].values

In [ ]:
fi_score(ans, pretict)

epoch = 3
train = 0.84 , test = 0.81

epoch = 6
train = 0.91 , test = 0.8526

epoch = 10
train = 0.94 , test = 0.849

epoch = 15
train = 0.97, test = 0.863

epoch = 20
train = 0.98, test = 0.859

In [ ]:
def fi_score(test, predict):

    if len(test) != len(predict):
        return print('예측과 정답의 길이가 다름')

    f1_df = [[0 for i in range(len(set(test)))] for i in range(len(set(test)))]

        
    for i in range(len(test)):
        colums = predict[i]
        idx =  test[i]
        f1_df[idx][colums] += 1

    recall = 0
    for i in range(len(set(test))):
        if sum(f1_df[i]) != 0 :
            recall +=f1_df[i][i]/sum(f1_df[i])
        else:
            recall += 0

    precision = 0
    for i in range(len(set(test))):
        
        total = 0
        for j in range(len(set(test))):
            total += f1_df[j][i]

        if total != 0 :
            precision +=f1_df[i][i]/total
            
        else:
            precision += 0
 
    recall = recall/6
    precision = precision/6
    f1_score = 2*(precision*recall)/(precision+recall)
    return f1_score

In [9]:
a = [i*1000 for i in range(1,27)]

In [10]:
a

[1000,
 2000,
 3000,
 4000,
 5000,
 6000,
 7000,
 8000,
 9000,
 10000,
 11000,
 12000,
 13000,
 14000,
 15000,
 16000,
 17000,
 18000,
 19000,
 20000,
 21000,
 22000,
 23000,
 24000,
 25000,
 26000]

In [11]:
sum(a)

351000